In [1]:
import pandas as pd
import numpy as np

In [11]:
eia_ferc = pd.read_csv('../datafiles/eia ferc fuzzy matched manually corrected.csv').dropna(subset='eia')
eia_codes = pd.read_csv('../datafiles/utilities_entity_eia.csv')
merged_eia_codes = pd.merge(eia_ferc,
        eia_codes,
        left_on='eia',
        right_on='utility_name_eia',
        how='left')
ferc_codes = pd.read_csv('../datafiles/utilities_ferc1.csv')
merged_ferc_codes = pd.merge(merged_eia_codes,
                            ferc_codes,
                            left_on='ferc',
                            right_on='utility_name_ferc1',
                            how='right')[['utility_id_eia',
                                         'utility_id_ferc1',
                                         'utility_name_ferc1',
                                         'utility_name_eia']]
states = pd.read_csv('../datafiles/utilities_eia860.csv',usecols=[0,2,5])
states['year_dt'] = pd.to_datetime(states['report_date']).dt.year
ferc_subset_with_state = pd.merge(states,
        merged_ferc_codes,
        how='right',
        on='utility_id_eia')
ferc_code_state_link = ferc_subset_with_state[['state','utility_id_ferc1']].dropna().drop_duplicates().sort_values(by='utility_id_ferc1')

In [4]:
power = pd.read_csv('denorm_purchased_power_ferc1.csv')
power_st = pd.merge(power,ferc_code_state_link,how='left',on='utility_id_ferc1')
power_st=power_st[power_st['state'].isin(['CA','WA','TX','MA'])]
power_st.to_csv('data_out/power purchase table.csv')

In [17]:
plant = pd.read_csv('denorm_plant_in_service_ferc1.csv')
plant_st = pd.merge(plant,ferc_code_state_link,how='left',on='utility_id_ferc1')
plant_st=plant_st[plant_st['state'].isin(['CA','WA','TX','MA'])]
plant_st = plant_st[plant_st['ferc_account_label'].isin(['transmission_plant','distribution_plant'])]
plant_st = plant_st[['report_year','utility_id_ferc1','utility_name_ferc1','state','additions','ferc_account_label']].dropna()
plant_st.to_csv('data_out/td plant additions.csv')

In [14]:
opex = pd.read_csv('denorm_electric_operating_expenses_ferc1.csv')
opex_st = pd.merge(opex,ferc_code_state_link,how='left',on='utility_id_ferc1')
opex_st=opex_st[opex_st['state'].isin(['CA','WA','TX','MA'])]


/var/folders/h0/rfb20rk90qz0tdgv08kqcyrw0000gn/T/ipykernel_57731/857445422.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  opex = pd.read_csv('denorm_electric_operating_expenses_ferc1.csv')


In [16]:
opex_st = opex_st[opex_st['expense_type'].isin(['transmission_expenses','distribution_expenses','injuries_and_damages'])]
opex_st = opex_st[['report_year','utility_id_ferc1','utility_name_ferc1','dollar_value','expense_type']].dropna()
opex_st.to_csv('data_out/t,d,injury opex.csv')